<a href="https://colab.research.google.com/github/majavk/EPRbiodose/blob/main/blind_dose_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pandas as pd
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
path = 'my_folder'
file_type = '.csv'

filenames = list()
for filename in os.listdir(path=path):
    if filename.endswith(file_type):
        filenames.append(filename)


In [ ]:
#one file in a folder is empty
MIN_ROW = 85
pd.read_csv(f'{path}/FT_No1_300_20052022_002.csv', header=None).iloc[MIN_ROW:, :]


Storing the spectral data in the data frame. The files acquired from the measurements are having 1st 85 rows filled with measurement condtitions.

In [ ]:
df_final = pd.DataFrame()

MIN_ROW = 85
for i, item in enumerate(filenames):
    variable_filename = '_' + item
    variable_name_to_change = item
    variable_name_to_change = pd.read_csv(f'{path}/' + item, header=None).iloc[MIN_ROW:, :]
    field_variable_name_to_change = list()
    ampl_variable_name_to_change = list()

    for  row in variable_name_to_change.itertuples():
      field_variable_name_to_change.append(float(row[1].split(';')[0]))
      ampl_variable_name_to_change.append(float(row[1].split(';')[1]))
      
    row_field_variable ='new_field' + str(variable_filename)
    row_ampl_variable = 'new_ampl' + str(variable_filename)
    try:
      df_final[row_field_variable] = field_variable_name_to_change
      df_final[row_ampl_variable] = ampl_variable_name_to_change
    except ValueError:
      print(f'The row {i} is having a Value Error')


The data of a spectra are separated in the first part where the standard, left, 3rd satelite of Mn2+ in MgO lies, for the field calibration data are and the second part where the data for the maximum of the EPR spectra of the sample response is.

In [ ]:
df_Mn_left = df_final[(df_final.iloc[:,0]> 330.) & (df_final.iloc[:, 0] < 333.)]
df_geff = df_final[(df_final.iloc[:,0]> 333.5) & (df_final.iloc[:, 0] < 338.)]

Separating field and amplitude spectral data for standard and sample signal.

In [ ]:
data = df_Mn_left.to_dict()
mn_left = pd.DataFrame()

for key, value in data.items():
    if 'ampl' in key:)
        B_MN_left_variable = 'B_Mn_left_'+key
        B_MN_left_column_variable = 'B_Mn_left_column_'+key
        values_Mn = 'values_Mn_'+ key
        values_Mn = list()
        for x,y in value.items():
            values_Mn.append(y)
        mn_left[B_MN_left_column_variable] = values_Mn
        

In [ ]:
data_1 = df_geff.to_dict()
df_geffective = pd.DataFrame()

for key, value in data_1.items():
    if 'ampl'in key:
        B_g_eff_variable = 'B_g_eff_'+key
        B_g_eff_column_variable = 'B_g_eff_column_'+key
        values = 'values_'+ key
        values = list()
        for x,y in value.items():
            values.append(y)
        df_geffective[B_g_eff_column_variable] = values


Evaluation of the g-values from the EPR spectral response (position of magnetic field where the absorption happened).

In [ ]:
def g_values(B_Mn_left, B_g_eff):
    g_Mn_left = 2.03339
    g_eff = g_Mn_left * (B_Mn_left/B_g_eff)
    return g_eff


Evaluation of magnetic field of maximum of EPR spectra for sample and the standard. 

In [ ]:
max_names = list()
max_values = list()
max_index = list()
for item in mn_left.columns.values.tolist():
    max_value = mn_left[item].sort_values(ascending=False)[:1]
    max_names.append(max_value.name)
    max_values.append(max_value.values[0])
    max_index.append(list(max_value.index)[0])
    
zipped_Mn = zip(max_names, max_values, max_index)
ampl_data_Mn = list(zipped_Mn)


In [ ]:
max_names_geff = list()
max_values_geff = list()
max_index_geff = list()

for item in df_geffective.columns.values.tolist():
    max_value = df_geffective[item].sort_values(ascending=False)[:1]
    max_names_geff.append(max_value.name)
    max_values_geff.append(max_value.values[0])
    max_index_geff.append(list(max_value.index)[0])
    
zipped_g_eff = zip(max_names_geff, max_values_geff, max_index_geff)
ampl_data_g_eff = list(zipped_g_eff)


In [ ]:
zipped_g_eff_Mn = zip(max_names, max_values, max_index, max_names_geff, max_values_geff, max_index_geff)
zipped_g_eff_Mn = list(zipped_g_eff_Mn)

Evaluation of the g_eff values and storing in the data frame.

In [ ]:
df = pd.DataFrame()

g_eff = list()
dose = list()
b_g_eff = list()
b_Mn_left = list()
for  name, value, index, name_geff, value_geff, index_geff in zipped_g_eff_Mn:
    replaced_name_geff = name_geff[15:].replace('ampl', 'field')
    replaced_name = name[17:].replace('ampl', 'field')
    B_g_eff = df_geff[replaced_name_geff].iloc[index_geff]
    B_Mn_left = df_Mn_left[replaced_name].iloc[index]
    variable_name = 'g_eff_' + replaced_name
    dose.append(variable_name)
    g_eff.append(g_values(B_Mn_left, B_g_eff))
    b_g_eff.append(B_g_eff)
    b_Mn_left.append(B_Mn_left)

df['g_eff'] = g_eff
df['meta_data'] = dose
df['B_g_eff'] = b_g_eff
df['B_Mn_left'] = b_Mn_left


Extracting the information on the dose, orientation and the sample from the column names.

In [ ]:
orientation = list()
sample = list()
for item in df['meta_data']:
  item = item.split('No')
  item = item[1][0:6]
  try:
    if item.endswith('_'):
      orientation.append(float(item[0:5].split('_')[1]))
      sample.append(float(item[0:5].split('_')[0]))
    else:
      orientation.append(float(item.split('_')[1]))
      sample.append(float(item.split('_')[0]))
  except IndexError:
    continue

df['No_sample'] = sample
df['orientation_sample'] = orientation
df_temp = df[['No_sample', 'orientation_sample', 'g_eff']]
df_temp = df_temp.sort_values('No_sample') 


In [ ]:
df_temp = df_temp.groupby('No_sample').mean()


By using previously established calibration curve, the dose evaluation from the g_eff values was performed.

In [ ]:
def reverse_cal(g_eff, y_0, a, b):
  return -1/b * np.log((a-g_eff+y_0)/a) 

In [ ]:
df_temp['dose'] = df_temp['g_eff'].apply(lambda x: reverse_cal(x, 2.0082, 0.0032, 0.524))
df = df_temp[['g_eff', 'dose']]